In [ ]:
import requests
import pandas as pd
import snowflake.snowpark as snowpark
from snowflake.snowpark.context import get_active_session

def main(session: snowpark.Session):
    # Step 1: Call the API
    url = "https://api.exchangerate-api.com/v4/latest/USD"
    response = requests.get(url)
    data = response.json()

    # Step 2: Transform data with Pandas
    df = pd.DataFrame(data['rates'].items(), columns=['currency', 'rate'])

    # Step 3: Write data into Snowflake
    session.write_pandas(df, table_name="exchange_rates", auto_create_table=True, overwrite=True)

In [ ]:
session = get_active_session()
main(session)

In [ ]:
# basic test UDF

import _snowflake
import requests
import logging

rsession = requests.Session()

def get_external_api_sp(api_url):
    try:
        access_token = _snowflake.get_oauth_access_token('some_token')
    except Exception as e:
        logging.error(f"Error: {str(e)}")
        return f"Error: {str(e)}"

    headers = {
        'Authorization': f'Bearer {access_token}'
    }

    try:
        response = requests.get(api_url, headers=headers)
        response.raise_for_status()
        logging.debug(f"Body from API: {response.text} in get external_api")
        if response.status_code != 200:
            logging.warning(f"Unexpected response from API: {response.status_code} in get external_api")
        data = response.json()
        return data
    except Exception as e:
        logging.error(f"Error: {str(e)}")
        return f"Error: {str(e)}"

In [ ]:
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.types import VariantType, StringType

session = get_active_session()

get_external_api_sp = session.udf.register(
    func=get_external_api_sp,
    return_type=VariantType(),
    input_types=[StringTypes()],
    name='get_external_api_sp',
    replace=True,
    is_permanent=True,
    stage_location='NB_TEST_STAG',
    secrets=[]
    external_access_integration=[""],
    packages=["snowflake-snowpark-python", "requests", "snowflake-telemetry-python"]
    
)

In [ ]:
import json
import datetime
import time
import logging

from snowflake.snowpark.types import VariantType, StringType, IntegerType, VariantType, BooleanType, TimestampType
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import call_function
from opentelemetry import trace

def extract_data_to_products(session):
    """
    Stored proc that extracts data from Sharepoint and loads to table
    """
    tracer = trace.get_tracer(__name__)

    # fetch columsn dynamically
    columns_df = session.sql("""
        SELECT COLUMN_NAME
        FROM INFORMATION_SCHEMA.COLUMNS
        WHERE TABLE_NAME = "PRODUCTS"
            AND TABLE_SCHEMA - "DATAPRODUCTS"
        ORDER_BY ORDINAL_POSITION
    """).collect()

    columns = [row['COLUMN_NAME'] for row in columns_df]

    session.sql("TRUNCATE TABKE PRODUCTS").collect()